##### Author: Prashant K. Sharma

In [48]:
import pandas as pd
import numpy as np
import io, sys
import json
import ast
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import glob

In [49]:
!pwd

/home/diptesh/workspace/PR-AAAI22-SDU-ST1-AE/nbs


In [50]:
langs = ["danish", "english", "french", "persian", "spanish", "vietnamese"]

## JSON to TSVs

In [51]:
danishDFTrain = pd.read_json("../data/"+langs[0]+"/train.json")
danishDFDev = pd.read_json("../data/"+langs[0]+"/dev.json")

englishDFTrainLegal = pd.read_json("../data/"+langs[1]+"/legal/train.json")
englishDFDevLegal = pd.read_json("../data/"+langs[1]+"/legal/dev.json")
englishDFTrainScientific = pd.read_json("../data/"+langs[1]+"/scientific/train.json")
englishDFDevScientific = pd.read_json("../data/"+langs[1]+"/scientific/dev.json")

frenchDFTrain = pd.read_json("../data/"+langs[2]+"/train.json")
frenchDFDev = pd.read_json("../data/"+langs[2]+"/dev.json")

persianDFTrain = pd.read_json("../data/"+langs[3]+"/train.json")
persianDFDev = pd.read_json("../data/"+langs[3]+"/dev.json")

spanishDFTrain = pd.read_json("../data/"+langs[4]+"/train.json")
spanishDFDev = pd.read_json("../data/"+langs[4]+"/dev.json")

vietnameseDFTrain = pd.read_json("../data/"+langs[5]+"/train.json")
vietnameseDFDev = pd.read_json("../data/"+langs[5]+"/dev.json")

In [52]:
englishDFTrainScientific.text[2], englishDFTrainScientific.acronyms[2]

(' An arguably better approach to representation learning is Canonical Correlation Analysis (CCA) that induces representations that are maximally cor-',
 [[91, 94]])

In [53]:
### Extracting text from the numerical indexes (Hadeel's Function)

In [54]:
# AUTHOR: Hadeel S

def create_acronyms(data):
    acs2 = []
    longf = []

    acros = data.acronyms.values
    txt = data.text.values
    zipped = list(zip(txt,acros))
    for z in zipped:
        acs = []
        for i in range(len(z[1])):
            acs.append(z[0][z[1][i][0]:z[1][i][1]])
        acs2.append(acs)
    data['acronyms-text'] =acs2
    
# Creating long forms from here

    longform = data["long-forms"].values
    zipped2 = list(zip(txt,longform))
    for z2 in zipped2:
        longform_temp = []
        for i in range(len(z2[1])):
            longform_temp.append(z2[0][z2[1][i][0]:z2[1][i][1]])
        longf.append(longform_temp)
    data["long-forms-text"]= longf
    
    return data
#     return data.to_csv("datatsvs/" + name +".tsv", index = False, encoding='utf8', sep="\t")

In [55]:
englishDFTrainScientific = create_acronyms(englishDFTrainScientific)
englishDFDevScientific = create_acronyms(englishDFDevScientific)
englishDFTrainLegal = create_acronyms(englishDFTrainLegal)
englishDFDevLegal = create_acronyms(englishDFDevLegal)
danishDFTrain = create_acronyms(danishDFTrain)
danishDFDev = create_acronyms(danishDFDev)
frenchDFTrain = create_acronyms(frenchDFTrain)
frenchDFDev = create_acronyms(frenchDFDev)
persianDFTrain = create_acronyms(persianDFTrain)
persianDFDev = create_acronyms(persianDFDev)
spanishDFTrain = create_acronyms(spanishDFTrain)
spanishDFDev = create_acronyms(spanishDFDev)
vietnameseDFTrain = create_acronyms(vietnameseDFTrain)
vietnameseDFDev = create_acronyms(vietnameseDFDev)

### Manipulating columns for proper TSVs

In [56]:
englishDFTrainScientific = englishDFTrainScientific[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
englishDFDevScientific = englishDFDevScientific[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
englishDFTrainLegal = englishDFTrainLegal[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
englishDFDevLegal = englishDFDevLegal[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
danishDFTrain = danishDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
danishDFDev = danishDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
frenchDFTrain = frenchDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
frenchDFDev = frenchDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
persianDFTrain = persianDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
persianDFDev = persianDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
spanishDFTrain = spanishDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
spanishDFDev = spanishDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
vietnameseDFTrain = vietnameseDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
vietnameseDFDev = vietnameseDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]

In [57]:
allLangDFTrain = englishDFTrainScientific.append(englishDFTrainLegal, ignore_index=True).append(danishDFTrain, ignore_index=True).append(frenchDFTrain, ignore_index=True).append(persianDFTrain, ignore_index=True).append(spanishDFTrain, ignore_index=True).append(vietnameseDFTrain, ignore_index=True)

In [58]:
allLangDFTrain.sample(frac=1)

,ID,text,acronyms,long-forms,acronyms-text,long-forms-text
23265,3521,La información que figura en la presente secci...,"[[672, 675]]","[[641, 670]]",[FMI],[Fondo Monetario Internacional]
695,696,are contained in a XML file and each query co...,"[[90, 93], [19, 22], [107, 112], [130, 134], [...","[[83, 88], [101, 106], [142, 151]]","[NUM, XML, TITLE, DESC, NARR]","[Numbe, Title, Narrative]"
21455,1711,"Entre 2000 y 2003, se realizaron tres evaluaci...","[[225, 229], [270, 273], [135, 140]]","[[181, 223], [236, 268]]","[OIEA, OMS, PNUMA]","[Organismo Internacional de Energía Atómica, O..."
15670,5045,Elle a rappelé que l'institution d'un barème d...,"[[459, 462]]","[[421, 457]]",[CAC],[Comité administratif de coordination]
17332,6707,Cette question occupe une place croissante dan...,"[[124, 127]]","[[84, 122]]",[OIT],[Organisation internationale du Travail]
...,...,...,...,...,...,...
23810,4066,Los organismos integrantes del Comité Ejecutiv...,"[[357, 360], [900, 906], [568, 575], [639, 645]]","[[317, 355], [833, 898], [460, 566], [581, 637]]","[OIT, ACNUDH, ONUSIDA, UNIFEM]","[Organización Internacional del Trabajo, Alto ..."
17869,7244,Costa Rica a également proposé la création d'u...,"[[422, 427]]","[[367, 420]]",[PNMPO],[Programa Nacional de Microcrédito Produtivo O...
2052,2053,"constituents. For example, discussing the poss...","[[190, 192]]","[[170, 188]]",[CG],[Categorial Grammar]
24372,4628,Desde que accedió a la independencia en 1921 h...,"[[334, 338]]","[[290, 332]]",[PRPM],[Partido Revolucionario Popular de Mongolia]


### Writing TSVs

In [59]:
englishDFTrainScientific.to_csv("../data_tsvs/englishDFTrainScientific.tsv", sep='\t', index=False)
englishDFDevScientific.to_csv("../data_tsvs/englishDFDevScientific.tsv", sep='\t', index=False)
englishDFTrainLegal.to_csv("../data_tsvs/englishDFTrainLegal.tsv", sep='\t', index=False)
englishDFDevLegal.to_csv("../data_tsvs/englishDFDevLegal.tsv", sep='\t', index=False)
danishDFTrain.to_csv("../data_tsvs/danishDFTrain.tsv", sep='\t', index=False)
danishDFDev.to_csv("../data_tsvs/danishDFDev.tsv", sep='\t', index=False)
frenchDFTrain.to_csv("../data_tsvs/frenchDFTrain.tsv", sep='\t', index=False)
frenchDFDev.to_csv("../data_tsvs/frenchDFDev.tsv", sep='\t', index=False)
persianDFTrain.to_csv("../data_tsvs/persianDFTrain.tsv", sep='\t', index=False)
persianDFDev.to_csv("../data_tsvs/persianDFDev.tsv", sep='\t', index=False)
spanishDFTrain.to_csv("../data_tsvs/spanishDFTrain.tsv", sep='\t', index=False)
spanishDFDev.to_csv("../data_tsvs/spanishDFDev.tsv", sep='\t', index=False)
vietnameseDFTrain.to_csv("../data_tsvs/vietnameseDFTrain.tsv", sep='\t', index=False)
vietnameseDFDev.to_csv("../data_tsvs/vietnameseDFDev.tsv", sep='\t', index=False)

In [60]:
allLangDFTrain.to_csv("../data_tsvs/allLangDFTrain.tsv", sep='\t', index=False)

# TSVs to CoNLL format 

In [61]:
def BIO(txt, an_ids, lf_ids):
    """
    input: text
    ouput: tags , bio_tags
        
        tags :    list (AN, FL, OT)
        bio_tags: list (O, B-AN, I-AN, B-LF, I-LF)
    """
    
    txt_l = word_tokenize(txt)
    
    tags = ["OT"]*len(txt_l)
    bio_tags = ["B-O"]*len(txt_l)
    
    #txt[an_ids[0][0]:an_ids[0][1]]
    
    ANs = []
    LFs = []
    
    for an_id in an_ids:
        ANs.append(txt[an_id[0]:an_id[1]])
        
    for lf_id in lf_ids:
        LFs.append(txt[lf_id[0]:lf_id[1]])
    
    AN_start = [False]*len(ANs)
    LF_start = [False]*len(LFs)
    
    
    for tok_id in range(0,len(txt_l)):
        tok = txt_l[tok_id]
        
        for AN_idx in range(0,len(ANs)):
           
            AN = ANs[AN_idx]
            
            if(tok in AN):
                tags[tok_id] = "AN"
                
                if(AN_start[AN_idx] == False):
                    AN_start[AN_idx] = True
                    bio_tags[tok_id] = "B-AN"
                else:
                    bio_tags[tok_id] = "I-AN"
        
        for LF_idx in range(0,len(LFs)):
            
            LF = LFs[LF_idx]
            
            if(tok in LF):
                tags[tok_id] = "LF"
                
                if(LF_start[LF_idx] == False):
                    LF_start[LF_idx] = True
                    bio_tags[tok_id] = "B-LF"
                else:
                    bio_tags[tok_id] = "I-LF"
                    
    return txt_l, tags, bio_tags

In [62]:
def extract(df, idx):
    df_i = df.iloc[idx]
    
    txt = str(df_i["text"]).strip()
    acrnm_ids = ast.literal_eval(df_i["acronyms"])
    longform_ids = ast.literal_eval(df_i["long-forms"])
    
    return txt, acrnm_ids, longform_ids

def prepare(df, cnt, fname="output.txt"):
    
    file = open(fname, "w")
    file.write("-DOCSTART- -X- O\n\n")
    for idx in tqdm(range(cnt)):
        txt, an_ids, lf_ids = extract(df, idx)
        
        txt_l, tags, bio_tags = BIO(txt, an_ids, lf_ids)
        
        for txt_id in range(0,len(txt_l)):
            string = txt_l[txt_id]+ " " + tags[txt_id] + " " + bio_tags[txt_id]
            
            file.write(string + " \n")
        file.write("\n")
    file.close()
    
def convert_to_BIO(fileTrain, fileDev, lang, domain):
    df_train = pd.read_csv(fileTrain, sep="\t")
    df_dev = pd.read_csv(fileDev, sep="\t")
    df_combine = df_train.append(df_dev, ignore_index=True)
    
    cnt_train = df_train.shape[0]
    cnt_dev = df_dev.shape[0]
    
    train_cnt = cnt_train - 500 ## Assuming 300 instances are enough to validate
    valid_cnt = 500
    test_cnt = cnt_dev
    
    df_train_ = df_combine.iloc[:train_cnt]
    df_valid_ = df_combine.iloc[train_cnt: train_cnt + valid_cnt].reset_index()
    df_test_ = df_combine.iloc[train_cnt + valid_cnt: train_cnt + valid_cnt + test_cnt].reset_index()
    
    # Prepare training file
    if ( domain == "" ):
        prepare(df_train_, train_cnt, "../processed/" + lang + "/train.txt") # Prepare train file
        prepare(df_valid_, valid_cnt, "../processed/" + lang + "/valid.txt") # Prepare validation file
        prepare(df_test_, test_cnt, "../processed/" + lang + "/test.txt") # Prepare test file
    else:
        prepare(df_train_, train_cnt, "../processed/" + lang + "/" + domain + "/train.txt") # Prepare train file
        prepare(df_valid_, valid_cnt, "../processed/" + lang + "/" + domain +"/valid.txt") # Prepare validation file
        prepare(df_test_, test_cnt, "../processed/" + lang+  "/" + domain + "/test.txt") # Prepare test file

In [63]:
convert_to_BIO("../data_tsvs/englishDFTrainLegal.tsv", "../data_tsvs/englishDFDevLegal.tsv", "eng", "legal")
convert_to_BIO("../data_tsvs/englishDFTrainScientific.tsv", "../data_tsvs/englishDFDevScientific.tsv", "eng", "scientific")
convert_to_BIO("../data_tsvs/danishDFTrain.tsv", "../data_tsvs/danishDFDev.tsv", "dan", "")
convert_to_BIO("../data_tsvs/frenchDFTrain.tsv", "../data_tsvs/frenchDFDev.tsv", "fre", "")
convert_to_BIO("../data_tsvs/persianDFTrain.tsv", "../data_tsvs/persianDFDev.tsv", "per", "")
convert_to_BIO("../data_tsvs/spanishDFTrain.tsv", "../data_tsvs/spanishDFDev.tsv", "esp", "")
convert_to_BIO("../data_tsvs/vietnameseDFTrain.tsv", "../data_tsvs/vietnameseDFDev.tsv", "vie", "")        

100%|██████████| 159/159 [00:00<00:00, 3265.84it/s]


In [47]:
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/englishDFDevLegal.tsv", "engAll", "legal")
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/englishDFDevScientific.tsv", "engAll", "scientific")
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/danishDFDev.tsv", "danAll", "")
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/frenchDFDev.tsv", "freAll", "")
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/persianDFDev.tsv", "perAll", "")
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/spanishDFDev.tsv", "espAll", "")
convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/vietnameseDFDev.tsv", "vieAll", "") 

100%|██████████| 159/159 [00:00<00:00, 3120.47it/s]
